In [1]:
import os
import sys
import pandas as pd
import numpy as np

In [ ]:
input_data_folder = "../input_annotations"
in_filename_csv = "test_log.csv"

In [ ]:
col_list = ["caseid", "task", "end_timestamp"]

In [ ]:
caseid_output = pd.read_csv(caseid_output_route, usecols=col_list)

In [ ]:
, usecols=col_list

In [ ]:
data = pd.read_csv(os.path.join(input_data_folder, in_filename_csv), sep=",")

In [ ]:
print(data[2:])

In [ ]:
data = pd.read_csv(os.path.join(input_data_folder, in_filename_csv), sep=",", usecols=col_list, dtype={'user': str})

In [ ]:
print(data[:2])

In [ ]:
def get_csv_events_data(data):
    """
    reads and parse all the events information from a csv file
    """
    log = pd.read_csv(input, dtype={'user': str})
    if one_timestamp:
        column_names['Complete Timestamp'] = 'end_timestamp'
        log = log.rename(columns=column_names)
        log = log.astype({'caseid': object})
        log = (log[(log.task != 'Start') & (log.task != 'End')]
               .reset_index(drop=True))
        if filter_d_attrib:
            log = log[['caseid', 'task', 'user', 'end_timestamp']]
        log['end_timestamp'] = pd.to_datetime(log['end_timestamp'],
                                              format=timeformat)
    else:
        column_names['Start Timestamp'] = 'start_timestamp'
        column_names['Complete Timestamp'] = 'end_timestamp'
        log = log.rename(columns=column_names)
        log = log.astype({'caseid': object})
        log = (log[(log.task != 'Start') & (log.task != 'End')]
               .reset_index(drop=True))
        if filter_d_attrib:
            log = log[['caseid', 'task', 'user',
                       'start_timestamp', 'end_timestamp']]
        log['start_timestamp'] = pd.to_datetime(log['start_timestamp'],
                                                format=timeformat)
        log['end_timestamp'] = pd.to_datetime(log['end_timestamp'],
                                              format=timeformat)
    log['user'].fillna('SYS', inplace=True)
    data = log.to_dict('records')
    append_csv_start_end()
    split_event_transitions()

In [ ]:
import itertools as it

In [ ]:
def append_csv_start_end(data):
    new_data = list()
    data = sorted(data, key=lambda x: x['caseid'])
    for key, group in it.groupby(data, key=lambda x: x['caseid']):
        trace = list(group)
        for new_event in ['Start', 'End']:
            idx = 0 if new_event == 'Start' else -1
            t_key = 'end_timestamp'
            if not one_timestamp and new_event == 'Start':
                t_key = 'start_timestamp'
            temp_event = dict()
            temp_event['caseid'] = trace[idx]['caseid']
            temp_event['task'] = new_event
            temp_event['user'] = new_event
            temp_event['end_timestamp'] = trace[idx][t_key]
            if not one_timestamp:
                temp_event['start_timestamp'] = trace[idx][t_key]
            if new_event == 'Start':
                trace.insert(0, temp_event)
            else:
                trace.append(temp_event)
        new_data.extend(trace)
    data = new_data
    return data

In [ ]:
def split_event_transitions(data, one_timestamp ):
    temp_raw = list()
    if one_timestamp:
        for event in data:
            temp_event = event.copy()
            temp_event['timestamp'] = temp_event.pop('end_timestamp')
            temp_event['event_type'] = 'complete'
            temp_raw.append(temp_event)
    else:
        for event in data:
            start_event = event.copy()
            complete_event = event.copy()
            start_event.pop('end_timestamp')
            complete_event.pop('start_timestamp')
            start_event['timestamp'] = start_event.pop('start_timestamp')
            complete_event['timestamp'] = complete_event.pop('end_timestamp')
            start_event['event_type'] = 'start'
            complete_event['event_type'] = 'complete'
            temp_raw.append(start_event)
            temp_raw.append(complete_event)
    data = temp_raw
    return data

In [ ]:
log = pd.read_csv(os.path.join(input_data_folder, in_filename_csv), sep=",", dtype={'user': str})
one_timestamp = True
timeformat = '%Y-%m-%dT%H:%M:%S.%f'
column_names = {'Case ID': 'caseid',
                'Activity': 'task',
                'lifecycle:transition': 'event_type', #---#
                'Resource': 'user'}
filter_d_attrib = {
    'timeformat': '%Y-%m-%dT%H:%M:%S.%f',
    'column_names': column_names,
    'one_timestamp': one_timestamp,
    'ns_include': True}
if one_timestamp:
    column_names['Complete Timestamp'] = 'end_timestamp'
    log = log.rename(columns=column_names)
    log = log.astype({'caseid': object})
    log = (log[(log.task != 'Start') & (log.task != 'End')]
           .reset_index(drop=True))
    if filter_d_attrib:
        log = log[['caseid', 'task', 'user', 'end_timestamp']]
    log['end_timestamp'] = pd.to_datetime(log['end_timestamp'],
                                          format=timeformat)
else:
    column_names['Start Timestamp'] = 'start_timestamp'
    column_names['Complete Timestamp'] = 'end_timestamp'
    log = log.rename(columns=column_names)
    log = log.astype({'caseid': object})
    log = (log[(log.task != 'Start') & (log.task != 'End')]
           .reset_index(drop=True))
    if filter_d_attrib:
        log = log[['caseid', 'task', 'user',
                   'start_timestamp', 'end_timestamp']]
    log['start_timestamp'] = pd.to_datetime(log['start_timestamp'],
                                            format=timeformat)
    log['end_timestamp'] = pd.to_datetime(log['end_timestamp'],
                                          format=timeformat)
log['user'].fillna('SYS', inplace=True)
data = log.to_dict('records')
data = append_csv_start_end(data)
data = split_event_transitions(data, one_timestamp)
data = pd

In [ ]:
print(data[:10])

In [ ]:
print(data[:1])

In [ ]:
data = pd.DataFrame.from_dict(data)

In [ ]:
print(data[:5])

In [ ]:
path = os.path.join(self.output_route,
                    'parameters',
                    'model_parameters.json')

if 'activity' in data:
    del data['activity']
self.parms = {**self.parms, **{k: v for k, v in data.items()}}
self.parms['dim'] = {k: int(v) for k, v in data['dim'].items()}
with open(path) as file:
    filter_parms_data = json.load(file)
    filter_parms['index_ac'] = {int(k): v
                              for k, v in data['index_ac'].items()}
    filter_parms['index_rl'] = {int(k): v
                              for k, v in data['index_rl'].items()}
    file.close()
    self.ac_index = {v: k for k, v in self.parms['index_ac'].items()}
    self.rl_index = {v: k for k, v in self.parms['index_rl'].items()}

In [ ]:
l = [[1,2,3,4],[5,6,7,8,9,10],[11,12,13]]

In [ ]:
key = l.index([5,6,7,8,9,10])

In [ ]:
print(key)

In [ ]:
len(l)

In [ ]:
dictA = {1:'Y', 2:'E', 3:'E'}
dictB = {"A":'Y', "B":'A', "C":'W'}

In [ ]:
dictB["A"] in dictA

In [ ]:
for name, age in dictA.items():
    if age == dictB["A"]:
        print(name)

In [ ]:
l

In [ ]:
l[1:]

In [ ]:
z = [[0], [0, 4.0], [0, 4.0, 6.0], [0, 4.0, 6.0, 3.0], [0, 4.0, 6.0, 3.0, 10.0], [0, 4.0, 6.0, 3.0, 10.0, 5.0], [0, 4.0, 6.0, 3.0, 10.0, 5.0, 8.0], [0, 4.0, 6.0, 3.0, 10.0, 5.0, 8.0, 7.0], [0, 4.0, 6.0, 3.0, 10.0, 5.0, 8.0, 7.0, 12.0]]

In [ ]:
z

In [ ]:
z[:5]

In [ ]:
import streamlit as st
from streamlit.hashing import _CodeHasher

try:
    # Before Streamlit 0.65
    from streamlit.ReportThread import get_report_ctx
    from streamlit.server.Server import Server
except ModuleNotFoundError:
    # After Streamlit 0.65
    from streamlit.report_thread import get_report_ctx
    from streamlit.server.server import Server


def main():
    state = _get_state()
    pages = {
        "Dashboard": page_dashboard,
        "Settings": page_settings,
    }

    st.sidebar.title(":floppy_disk: Page states")
    page = st.sidebar.radio("Select your page", tuple(pages.keys()))

    # Display the selected page with the session state
    pages[page](state)

    # Mandatory to avoid rollbacks with widgets, must be called at the end of your app
    state.sync()


def page_dashboard(state):
    st.title(":chart_with_upwards_trend: Dashboard page")
    display_state_values(state)


def page_settings(state):
    st.title(":wrench: Settings")
    display_state_values(state)

    st.write("---")
    options = ["Hello", "World", "Goodbye"]
    state.input = st.text_input("Set input value.", state.input or "")
    state.slider = st.slider("Set slider value.", 1, 10, state.slider)
    state.radio = st.radio("Set radio value.", options, options.index(state.radio) if state.radio else 0)
    state.checkbox = st.checkbox("Set checkbox value.", state.checkbox)
    state.selectbox = st.selectbox("Select value.", options, options.index(state.selectbox) if state.selectbox else 0)
    state.multiselect = st.multiselect("Select value(s).", options, state.multiselect)

    # Dynamic state assignments
    for i in range(3):
        key = f"State value {i}"
        state[key] = st.slider(f"Set value {i}", 1, 10, state[key])


def display_state_values(state):
    st.write("Input state:", state.input)
    st.write("Slider state:", state.slider)
    st.write("Radio state:", state.radio)
    st.write("Checkbox state:", state.checkbox)
    st.write("Selectbox state:", state.selectbox)
    st.write("Multiselect state:", state.multiselect)
    
    for i in range(3):
        st.write(f"Value {i}:", state[f"State value {i}"])

    if st.button("Clear state"):
        state.clear()


class _SessionState:

    def __init__(self, session, hash_funcs):
        """Initialize SessionState instance."""
        self.__dict__["_state"] = {
            "data": {},
            "hash": None,
            "hasher": _CodeHasher(hash_funcs),
            "is_rerun": False,
            "session": session,
        }

    def __call__(self, **kwargs):
        """Initialize state data once."""
        for item, value in kwargs.items():
            if item not in self._state["data"]:
                self._state["data"][item] = value

    def __getitem__(self, item):
        """Return a saved state value, None if item is undefined."""
        return self._state["data"].get(item, None)
        
    def __getattr__(self, item):
        """Return a saved state value, None if item is undefined."""
        return self._state["data"].get(item, None)

    def __setitem__(self, item, value):
        """Set state value."""
        self._state["data"][item] = value

    def __setattr__(self, item, value):
        """Set state value."""
        self._state["data"][item] = value
    
    def clear(self):
        """Clear session state and request a rerun."""
        self._state["data"].clear()
        self._state["session"].request_rerun()
    
    def sync(self):
        """Rerun the app with all state values up to date from the beginning to fix rollbacks."""

        # Ensure to rerun only once to avoid infinite loops
        # caused by a constantly changing state value at each run.
        #
        # Example: state.value += 1
        if self._state["is_rerun"]:
            self._state["is_rerun"] = False
        
        elif self._state["hash"] is not None:
            if self._state["hash"] != self._state["hasher"].to_bytes(self._state["data"], None):
                self._state["is_rerun"] = True
                self._state["session"].request_rerun()

        self._state["hash"] = self._state["hasher"].to_bytes(self._state["data"], None)


def _get_session():
    session_id = get_report_ctx().session_id
    session_info = Server.get_current()._get_session_info(session_id)

    if session_info is None:
        raise RuntimeError("Couldn't get your Streamlit Session object.")
    
    return session_info.session


def _get_state(hash_funcs=None):
    session = _get_session()

    if not hasattr(session, "_custom_session_state"):
        session._custom_session_state = _SessionState(session, hash_funcs)

    return session._custom_session_state


if __name__ == "__main__":
    main()

In [ ]:
n = [0]

In [ ]:
n = n + 1

In [ ]:
u = 4

In [ ]:
y = [u-1]

In [ ]:
parms['multiprednum']

In [ ]:
from num2words import num2words as nw

In [ ]:
nw(1, lang="en", to="ordinal_num")

In [ ]:
session_state = {'pos_tm_ss': [[0], [0.0002507188]], 
'initial_prediction': {'ss_initpredict1': {'pos_ac_ss': [], 'pos_rl_ss': [0, 2], 'pos_lb_ss': [0, 1]}, 
						'ss_initpredict2': {'pos_ac_ss': [0, 10], 'pos_rl_ss': [0, 4], 'pos_lb_ss': [0, 0]}, 
						'ss_initpredict3': {'pos_ac_ss': [0, 3], 'pos_rl_ss': [0, 5], 'pos_lb_ss': [0, 0]}}}

In [ ]:
session_state 

In [ ]:
_hist_columns = ['pos_ac_ss', 'pos_rl_ss', 'pos_lb_ss']

In [ ]:
for lk in range(3):
    _hist_predicted_dict_arl = dict([(k, session_state['initial_prediction']['ss_initpredict'+str(lk+1)][k]) for k in _hist_columns ])

In [ ]:
_hist_predicted_dict_arl

In [ ]:
_hist_predicted_dict_arl.update(dict([(k, session_state[k]) for k in ['pos_tm_ss']]))

In [ ]:
_hist_predicted_dict_arl

In [ ]:
if session_state['initial_prediction']['ss_initpredict1']['pos_ac_ss'] != []:
    print('hello')

In [ ]:
z = ['SME', 'Prediction 1', 'Prediction 2', 'Prediction 3']

In [ ]:
z

In [ ]:
z[0]

In [ ]:
for kz in range(3+1):
    x = z[kz]
    print(x)

In [ ]:
session_ = {}

In [ ]:
session_['history_of_choice'] = dict()

In [ ]:
session_['history_of_choice'] = {'hist_ac_prefix': [], 'hist_rl_prefix': [], 'hist_lb_prefix': [], 'hist_tm_prefix': [], 'hist_pred_prefix': []}

In [ ]:
session_

In [ ]:
a = [0, 4, 5, 6]

In [ ]:
zx = [np.array([0.00025072]), np.array([0.38379776])]

In [ ]:
zx

In [ ]:
session_state['multi_pred_ss'] = dict()

In [ ]:
for _zx in range(3):
    session_state['multi_pred_ss']["ss_multipredict{0}".format(_zx + 1)] = {}
    for _ux in range(3):
        session_state['multi_pred_ss']["ss_multipredict" + str(_zx + 1)]["multiverse_predict{0}".format(_ux + 1)] = {'ac_pred': [], 'ac_prob': [], 'rl_pred': [], 'rl_prob': [], 'label_pred': [], 'label_prob': [], 'tm_pred': []}

In [ ]:
session_state

In [ ]:
session_state['multi_pred_ss']['ss_multipredict1']['multiverse_predict1']

In [ ]:
a = [1, 2]

In [ ]:
[a[0]]

In [ ]:
import random

In [ ]:
[random.choice(a)]

In [ ]:
a.extend([None])

In [ ]:
a

In [1]:
x = input()

leyla


In [2]:
print("Hello ", x)

Hello  leyla


In [3]:
a = [1, 2, 3, 4, 5, 6]

In [7]:
a[0:1]

[1]

In [8]:
a.iloc[:-1]

AttributeError: 'list' object has no attribute 'iloc'

In [2]:
import numpy as np
import pandas as pd
df = pd.DataFrame(np.random.randint(0,100,size=(10, 4)), columns=list('ABCD'))

In [6]:
df

,A,B,C,D
0,74,53,20,45
1,89,54,35,94
2,51,64,16,61
3,43,78,94,70
4,32,53,59,99
5,23,3,25,56
6,71,2,99,7
7,34,83,88,46
8,65,99,69,49
9,0,59,63,2


In [7]:
df.columns

Index(['A', 'B', 'C', 'D'], dtype='object')

In [9]:
df[['A']]

,A
0,74
1,89
2,51
3,43
4,32
5,23
6,71
7,34
8,65
9,0


In [10]:
df.rename(columns={"A": "aa", "B": "cc"})

,aa,cc,C,D
0,74,53,20,45
1,89,54,35,94
2,51,64,16,61
3,43,78,94,70
4,32,53,59,99
5,23,3,25,56
6,71,2,99,7
7,34,83,88,46
8,65,99,69,49
9,0,59,63,2


In [13]:
sum(df[['A']].values.tolist(), [])

[74, 89, 51, 43, 32, 23, 71, 34, 65, 0]

In [14]:
df

,A,B,C,D
0,74,53,20,45
1,89,54,35,94
2,51,64,16,61
3,43,78,94,70
4,32,53,59,99
5,23,3,25,56
6,71,2,99,7
7,34,83,88,46
8,65,99,69,49
9,0,59,63,2


In [17]:
df.iloc[-1:].index

RangeIndex(start=9, stop=10, step=1)

In [23]:
tm = [array([0.]), array([0.]), array([970.]), array([24.]), array([16.]), array([0.]), array([0.]), array([307.]), array([14643.]), array([50.]), array([2955.]), array([52792.]), array([0.]), array([0.]), array([86400.]), array([172800.]), array([36000.])] 

NameError: name 'array' is not defined

In [38]:
arr1 = np.arange(4)
arr2 = np.arange(5,7)
arr3 = np.arange(7,12)

array_of_arrays = np.array([arr1, arr2, arr3])
array_of_arrays
np.concatenate(array_of_arrays)

C:\Users\Rehan\anaconda3\envs\lstm_exp_v3\lib\site-packages\ipykernel_launcher.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """


array([ 0,  1,  2,  3,  5,  6,  7,  8,  9, 10, 11])

In [39]:
tmz = [np.array([0]), np.array([1]), np.array([2]), np.array([3])]

In [40]:
tmz

[array([0]), array([1]), array([2]), array([3])]

In [36]:
z = sum(tmz, []) 

In [37]:
z

array([], dtype=float64)

[[0], [1], [2], [3]]

In [46]:
z = sum([x.tolist() for x in tmz], []) 

In [47]:
z

[0, 1, 2, 3]

In [59]:
df

,A,B,C,D
0,36,37,28,89
1,36,25,68,27
2,27,79,97,63
3,71,10,95,59
4,42,35,84,64
5,20,83,99,3
6,62,61,19,25
7,96,77,93,44
8,1,50,42,95
9,36,55,43,36


In [70]:
group = df.sort_values('A', ascending=False, kind='mergesort')

In [71]:
group

,A,B,C,D
7,96,77,93,44
3,71,10,95,59
6,62,61,19,25
4,42,35,84,64
0,36,37,28,89
1,36,25,68,27
9,36,55,43,36
2,27,79,97,63
5,20,83,99,3
8,1,50,42,95


In [63]:
df.columns

Index(['A', 'B', 'C', 'D'], dtype='object')

In [72]:
df['A']

0    36
1    36
2    27
3    71
4    42
5    20
6    62
7    96
8     1
9    36
Name: A, dtype: int32

In [74]:
df['A'].shift(-1)

0    36.0
1    27.0
2    71.0
3    42.0
4    20.0
5    62.0
6    96.0
7     1.0
8    36.0
9     NaN
Name: A, dtype: float64

In [ ]:
def check_if_activity_exists_and_time_less_than(group, activity):
    relevant_activity_idxs = np.where(group[activity_col] == activity)[0]
    if len(relevant_activity_idxs) > 0:
        idx = relevant_activity_idxs[0]
        if group["timesincelastevent"].iloc[idx] <= 28 * 1440: # return in less than 28 days (converted in minutes)
            group[label_col] = pos_label #deviant
            return group[:idx] #removes the activity in the scope
        else:
            group[label_col] = neg_label #regular
            return group[:idx]
    else:
        group[label_col] = neg_label #regular
        return group

In [ ]:
def check_if_activity_exists(group, activity):
    relevant_activity_idxs = np.where(group[activity_col] == activity)[0]
    if len(relevant_activity_idxs) > 0:
        idx = relevant_activity_idxs[0]
        group[label_col] = pos_label #deviant
        return group[:idx]
    else:
        group[label_col] = neg_label #regular
        return group

In [75]:
y = {"file_name": "sepsis_cases.csv", "imp": 2}

In [76]:
y

{'file_name': 'sepsis_cases.csv', 'imp': 2}

In [83]:
"sepsis" in y['file_name']

True

In [81]:
y.values()

dict_values(['sepsis_cases.csv', 2])

In [2]:
f = [['CRP'], ['CRP'], ['LacticAcid'], ['LacticAcid'], ['CRP'], ['CRP'], ['IV Liquid'], ['IV Antibiotics'], ['Admission NC'], ['Admission NC'], ['Release A'], ['CRP'], ['Release A'], ['CRP'], ['Release A'], ['CRP']]

In [3]:
f

[['CRP'],
 ['CRP'],
 ['LacticAcid'],
 ['LacticAcid'],
 ['CRP'],
 ['CRP'],
 ['IV Liquid'],
 ['IV Antibiotics'],
 ['Admission NC'],
 ['Admission NC'],
 ['Release A'],
 ['CRP'],
 ['Release A'],
 ['CRP'],
 ['Release A'],
 ['CRP']]

In [4]:
sum(f, []) 

['CRP',
 'CRP',
 'LacticAcid',
 'LacticAcid',
 'CRP',
 'CRP',
 'IV Liquid',
 'IV Antibiotics',
 'Admission NC',
 'Admission NC',
 'Release A',
 'CRP',
 'Release A',
 'CRP',
 'Release A',
 'CRP']

In [8]:
m = ['Herb', 'Alec', 'Herb', 'Don']

In [9]:
z = sorted(set(m), key=m.index)

In [10]:
z

['Herb', 'Alec', 'Don']